In [62]:
import logging
import argparse
import os
import numpy as np
import torch

from ppnp.pytorch import PPNP
from ppnp.pytorch.training import train_model
from ppnp.pytorch.earlystopping import stopping_args
from ppnp.pytorch.propagation import PPRExact, PPRPowerIteration
from ppnp.data.io import load_dataset

In [63]:
logging.basicConfig(
        format='%(asctime)s: %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO + 2)

# PyTorch variant
This is not the official code that should be used to reproduce the results from the paper (see `reproduce_results.ipynb`, available at https://github.com/russellizadi/ssp for this), but an adaptation of that code to PyTorch for better accessibility.

To run this notebook, one first needs to download the paper's code at https://github.com/russellizadi/ssp.

# Load dataset

In [64]:
graph_name = 'pubmed'
graph = load_dataset(graph_name)
graph.standardize(select_lcc=True)

<Undirected, unweighted and connected SparseGraph with 88648 edges (no self-loops). Data: adj_matrix (19717x19717), attr_matrix (19717x500), labels (19717)>

# Set up data splits

First of all we need to decide whether to use the test or validation set. Be mindful that we can only look at the test set exactly _once_ and then can't change any hyperparameters oder model details, no matter what. Everything else would cause overfitting.

In [65]:
test = True

These are the seeds for the dataset splits used in the paper for test/validation.

In [66]:
test_seeds = [
        2144199730,  794209841, 2985733717, 2282690970, 1901557222,
        2009332812, 2266730407,  635625077, 3538425002,  960893189,
        497096336, 3940842554, 3594628340,  948012117, 3305901371,
        3644534211, 2297033685, 4092258879, 2590091101, 1694925034]
val_seeds = [
        2413340114, 3258769933, 1789234713, 2222151463, 2813247115,
        1920426428, 4272044734, 2092442742, 841404887, 2188879532,
        646784207, 1633698412, 2256863076,  374355442,  289680769,
        4281139389, 4263036964,  900418539,  119332950, 1628837138]

if test:
    seeds = test_seeds
else:
    seeds = val_seeds

Now we can choose the remaining settings for the training/early stopping/validation(test) split. These are the ones chosen in the paper

In [67]:
if graph_name == 'microsoft_academic':
    nknown = 5000
else:
    nknown = 1500
    
idx_split_args = {'ntrain_per_class': 20, 'nstopping': 500, 'nknown': nknown}

# Set up propagation

Next we need to set up the proper pmropagation scheme. In the paper we've introduced the exact PPR propagation used in PPNP and the PPR power iteration propagation used in APPNP.

We use the hyperparameters from the paper.

In [68]:
if graph_name == 'microsoft_academic':
    alpha = 0.2
else:
    alpha = 0.1

prop_ppnp = PPRExact(graph.adj_matrix, alpha=alpha)
prop_appnp = PPRPowerIteration(graph.adj_matrix, alpha=alpha, niter=10)

# Choose model hyperparameters

Now we choose the hyperparameters. These are the ones used in the paper for all datasets.

Note that we choose the propagation for APPNP.

In [69]:
model_args = {
    'hiddenunits': [64],
    'drop_prob': 0.5,
    'propagation': prop_appnp}

reg_lambda = 5e-3
learning_rate = 0.01

# Train model

First we set the remaining settings for training.

In [70]:
niter_per_seed = 1
save_result = False
print_interval = 100
device = 'cpu'

We use 20 different seeds for splitting and 5 iterations (different random initializations) per split, so we train 100 times altogether. This will take a while.

In [71]:
results = []
niter_tot = niter_per_seed * len(seeds)
i_tot = 0
for seed in seeds:
    idx_split_args['seed'] = seed
    for _ in range(niter_per_seed):
        i_tot += 1
        logging_string = f"Iteration {i_tot} of {niter_tot}"
        logging.log(22,
                logging_string + "\n                     "
                + '-' * len(logging_string))
        _, result = train_model(
            graph_name, PPNP, graph, model_args, learning_rate, reg_lambda,
            idx_split_args, stopping_args, test, device, None, print_interval)
        results.append({})
        results[-1]['stopping_accuracy'] = result['early_stopping']['accuracy']
        results[-1]['valtest_accuracy'] = result['valtest']['accuracy']
        results[-1]['runtime'] = result['runtime']
        results[-1]['runtime_perepoch'] = result['runtime_perepoch']
        results[-1]['split_seed'] = seed

2023-04-19 15:02:41: Iteration 1 of 20
                     -----------------
2023-04-19 15:02:41: PyTorch seed: 3026156784
2023-04-19 15:04:35: Last epoch: 768, best epoch: 496 (114.100 sec)
2023-04-19 15:04:36: Test accuracy: 79.8%
2023-04-19 15:04:36: Iteration 2 of 20
                     -----------------
2023-04-19 15:04:36: PyTorch seed: 1137220691
2023-04-19 15:07:01: Last epoch: 988, best epoch: 762 (145.307 sec)
2023-04-19 15:07:01: Test accuracy: 74.6%
2023-04-19 15:07:01: Iteration 3 of 20
                     -----------------
2023-04-19 15:07:01: PyTorch seed: 3073648422
2023-04-19 15:09:02: Last epoch: 814, best epoch: 393 (120.223 sec)
2023-04-19 15:09:02: Test accuracy: 77.5%
2023-04-19 15:09:02: Iteration 4 of 20
                     -----------------
2023-04-19 15:09:02: PyTorch seed: 3098751611
2023-04-19 15:10:50: Last epoch: 721, best epoch: 621 (107.410 sec)
2023-04-19 15:10:50: Test accuracy: 77.9%
2023-04-19 15:10:50: Iteration 5 of 20
                     ----

# Evaluation

To evaluate the data we use Pandas and Seaborn (for bootstrapping).

In [72]:
import pandas as pd
import seaborn as sns

In [73]:
result_df = pd.DataFrame(results)
result_df.head()

,stopping_accuracy,valtest_accuracy,runtime,runtime_perepoch,split_seed
0,0.806,0.798156,114.100091,0.148375,2144199730
1,0.770,0.745732,145.306779,0.146923,794209841
2,0.782,0.774606,120.223032,0.147513,2985733717
3,0.762,0.779053,107.410031,0.148767,2282690970
4,0.800,0.795356,169.568509,0.149928,1901557222


The standard deviation doesn't really say much about the uncertainty of our results and the standard error of the mean (SEM) assumes a normal distribution. So the best way to get a valid estimate for our results' uncertainty is via bootstrapping.

In [74]:
def calc_uncertainty(values: np.ndarray, n_boot: int = 1000, ci: int = 95) -> dict:
    stats = {}
    stats['mean'] = values.mean()
    boots_series = sns.algorithms.bootstrap(values, func=np.mean, n_boot=n_boot)
    stats['CI'] = sns.utils.ci(boots_series, ci)
    stats['uncertainty'] = np.max(np.abs(stats['CI'] - stats['mean']))
    return stats

In [75]:
stopping_acc = calc_uncertainty(result_df['stopping_accuracy'])
valtest_acc = calc_uncertainty(result_df['valtest_accuracy'])
runtime = calc_uncertainty(result_df['runtime'])
runtime_perepoch = calc_uncertainty(result_df['runtime_perepoch'])

In [76]:
print("APPNP\n"
      "Early stopping: Accuracy: {:.2f} ± {:.2f}%\n"
      "{}: Accuracy: {:.2f} ± {:.2f}%\n"
      "Runtime: {:.3f} ± {:.3f} sec, per epoch: {:.2f} ± {:.2f}ms"
      .format(
          stopping_acc['mean'] * 100,
          stopping_acc['uncertainty'] * 100,
          'Test' if test else 'Validation',
          valtest_acc['mean'] * 100,
          valtest_acc['uncertainty'] * 100,
          runtime['mean'],
          runtime['uncertainty'],
          runtime_perepoch['mean'] * 1e3,
          runtime_perepoch['uncertainty'] * 1e3,
      ))

APPNP
Early stopping: Accuracy: 78.64 ± 1.25%
Test: Accuracy: 78.47 ± 0.90%
Runtime: 159.737 ± 59.436 sec, per epoch: 163.58 ± 34.30ms
